In [1]:
import requests
import pandas as pd

pd.set_option('display.max_rows', None)

Given a ticker, we will query the
- stock price
- available exercise prices
- available times to expiration
- risk-free interest rate
- volatility

and use black scholes formula to return the option price

Choosing Options:

- Choose Ticker
> see all expiries
- Choose Expiry
> See options chain for that Expiry
- Choose option by code
- Plug it into formula

or

- Choose Ticker, OptType, Expiry and Strike
- Plug into formula

In [2]:
API_KEY = "OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"

MEMORY = {}

OPTIONS = {}

RENAME_MAP = {
    'CALL': {
        'strike': 'Strike', 
        'contractName': 'cName',
        'lastPrice': 'lastPrice', 
        'bid': 'cBidP', 
        'ask': 'cAskP',
        'volume': 'cVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'cDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    },
    'PUT': {
        'strike': 'Strike', 
        'contractName': 'pName',
        'lastPrice': 'lastPrice', 
        'bid': 'pBidP', 
        'ask': 'pAskP',
        'volume': 'pVolume', 
        'impliedVolatility': 'impliedVolatility', 
        'delta': 'pDelta', 
        'gamma': 'Gamma', 
        'theta': 'Theta', 
        'vega': 'Vega', 
        'rho': 'Rho'
    }
}



In [3]:
def query_data(ticker):
    global OPTIONS
    global MEMORY
    global API_KEY
    global RENAME_MAP

    if ticker not in OPTIONS:
        url = f"https://eodhistoricaldata.com/api/options/{ticker}.US?api_token={API_KEY}"
        response = requests.request("GET", url)

        OPTIONS[ticker] = response.json()
        OPTIONS[ticker]['expiries'] = [i['expirationDate'] for i in OPTIONS[ticker]['data']]
        OPTIONS[ticker]['options_chain'] = {}

        for sample in OPTIONS[ticker]['data']:
            opt_chain = {opt['strike']:{} for opt in sample['options']["CALL"]} if len(sample['options']["CALL"]) > len(sample['options']["PUT"]) else {opt['strike']:{} for opt in sample['options']["PUT"]}
            opt_types = ['CALL', 'PUT']
            for opt_type in opt_types:
                for opt in sample['options'][opt_type]: 
                    opt_chain[opt['strike']].update({RENAME_MAP[opt_type][k]:v for k, v in opt.items() if k in ['strike', 'bid', 'ask', 'volume', 'delta', 'gamma', 'theta', 'vega', 'rho', 'contractName']})
                    MEMORY[opt['contractName']] = opt
            
            OPTIONS[ticker]['options_chain'][sample['expirationDate']] = pd.DataFrame(list(opt_chain.values()), columns = ["Theta", "Gamma", "Rho", "Vega", "cName", "cDelta", "cVolume", "cBidP", "cTheo", "cAskP", "Strike", "pBidP", "pTheo", "pAskP", "pVolume", "pDelta", 'pName'])
    
    return OPTIONS[ticker]['data']

class OptionFromId:
    def __init__(self, contract_name, memory={}):
        self.contract_name = contract_name
        self.ticker = ""
        self.generate_ticker()

        query_data(self.ticker)

        if memory:
            self.memory = {contract_name: memory}
        else:
            self.memory = MEMORY
        
        self.stock_price = OPTIONS[self.ticker]['lastTradePrice']

        self.opt_type = self.memory[contract_name]['type']
        self.exercise_price =  self.memory[contract_name]['strike']
        self.time_to_expiration =  self.memory[contract_name]['daysBeforeExpiration']

        self.volatility = 0
        self.interest_rate = 0

    def generate_ticker(self):
        for i in self.contract_name:
            if i.isalpha():
                self.ticker += i
            else: break

class OptionByChoice:
    def __init__(self, ticker, opt_type, strike, expiration):
        self.data = query_data(ticker)
        self.strike = strike
        self.expiration = expiration

        opts_date = self.find_opt_date()
        self.opt_data = self.find_opt(opts_date['options'][opt_type])
        self.option = OptionFromId(self.opt_data['contractName'], memory=self.opt_data)

    def find_opt_date(self):
        for opt in self.data:
            if opt["expirationDate"] == self.expiration:
                return opt
        raise KeyError(f"No option with date: {self.expiration}")

    def find_opt(self, opts_datetype):
        for opt in opts_datetype:
            if opt['strike'] == self.strike:
                return opt
        raise KeyError(f"No option with date: {self.expiration}, strike: {self.strike}")

In [10]:
config = {
    'ticker': "AAPL"
}

# or 

# config = {
#     'ticker': "AAPL",
#     'opt_type': 'CALL',
#     'strike': 170,
#     'expiration': '2022-02-25'
# }

# or

# config = {
#     'id': "AAPL220218C00070000"
# }

class OptionsPricer:
    def __init__(self, config):
        self.config = config
        if {'opt_type', 'strike', 'expiration'}.issubset(set(config.keys())):
            self.option = OptionByChoice(**config).option
        elif 'id' in config.keys():
            self.option = OptionFromId(config['id'])
        else:
            query_data(config['ticker'])
            print("See below expiries, and then `OPTIONS[<ticker>]['options_chain'][<expiry>]` to see options chains")
            print(OPTIONS[config['ticker']]['expiries'])

aapl = OptionsPricer(config)
# aapl.option

See below expiries, and then `OPTIONS[option_chain][<expiry>]` to see options chains
['2022-02-18', '2022-02-25', '2022-03-04', '2022-03-11', '2022-03-18', '2022-03-25', '2022-04-01', '2022-04-14', '2022-05-20', '2022-06-17', '2022-07-15', '2022-08-19', '2022-09-16', '2022-10-21', '2022-11-18', '2023-01-20', '2023-03-17', '2023-06-16', '2023-09-15', '2024-01-19']


In [14]:
OPTIONS["AAPL"]["options_chain"]['2022-03-25']

,Theta,Gamma,Rho,Vega,cName,cDelta,cVolume,cBidP,cTheo,cAskP,Strike,pBidP,pTheo,pAskP,pVolume,pDelta,pName
0,-0.0106,0.0006,0,0.0112,AAPL220325C00105000,0.9845,NaN,63.75,NaN,64.40,105,0.06,NaN,0.13,4.0,-0.0078,AAPL220325P00105000
1,-0.0130,0.0009,0,0.0146,AAPL220325C00110000,0.9804,64.0,58.80,NaN,59.45,110,0.10,NaN,0.15,4.0,-0.0105,AAPL220325P00110000
2,-0.0160,0.0012,0,0.0190,AAPL220325C00115000,0.9848,NaN,53.70,NaN,54.35,115,0.14,NaN,0.19,1.0,-0.0143,AAPL220325P00115000
3,-0.0176,0.0015,0,0.0226,AAPL220325C00120000,0.9720,NaN,48.90,NaN,49.50,120,0.19,NaN,0.24,3.0,-0.0175,AAPL220325P00120000
4,-0.0232,0.0022,0,0.0313,AAPL220325C00125000,0.9507,NaN,42.90,NaN,46.20,125,0.27,NaN,0.33,13.0,-0.0257,AAPL220325P00125000
5,-0.0297,0.0031,0,0.0421,AAPL220325C00130000,0.9655,2.0,38.85,NaN,39.55,130,0.37,NaN,0.44,16.0,-0.0368,AAPL220325P00130000
6,-0.0379,0.0044,0,0.0566,AAPL220325C00135000,0.9401,NaN,33.30,NaN,35.75,135,0.52,NaN,0.59,16.0,-0.0531,AAPL220325P00135000
7,-0.0456,0.0061,0,0.0728,AAPL220325C00140000,0.9369,1.0,28.70,NaN,30.20,140,0.73,NaN,0.80,190.0,-0.0734,AAPL220325P00140000
8,-0.0558,0.0084,0,0.0946,AAPL220325C00145000,0.8883,1.0,24.80,NaN,25.40,145,1.06,NaN,1.16,50.0,-0.1042,AAPL220325P00145000
9,-0.0676,0.0114,0,0.1212,AAPL220325C00150000,0.8455,40.0,20.30,NaN,20.90,150,1.54,NaN,1.66,225.0,-0.1487,AAPL220325P00150000


In [99]:
MEMORY

{'AAPL220218C00070000': {'contractName': 'AAPL220218C00070000',
  'contractSize': 'REGULAR',
  'contractPeriod': 'MONTHLY',
  'currency': 'USD',
  'type': 'CALL',
  'inTheMoney': 'TRUE',
  'lastTradeDateTime': '2022-02-14 15:55:06',
  'expirationDate': '2022-02-18',
  'strike': 70,
  'lastPrice': 98.2,
  'bid': 102.3,
  'ask': 102.75,
  'change': -4.23,
  'changePercent': -0.0413,
  'volume': 1,
  'openInterest': 16,
  'impliedVolatility': 549.0118,
  'delta': 0.9923,
  'gamma': 0.0003,
  'theta': -0.3696,
  'vega': 0.0027,
  'rho': 0,
  'theoretical': 102.75,
  'intrinsicValue': 0,
  'timeValue': 0,
  'updatedAt': '2022-02-16 19:20:05',
  'daysBeforeExpiration': 1},
 'AAPL220218C00075000': {'contractName': 'AAPL220218C00075000',
  'contractSize': 'REGULAR',
  'contractPeriod': 'MONTHLY',
  'currency': 'USD',
  'type': 'CALL',
  'inTheMoney': 'TRUE',
  'lastTradeDateTime': '2022-02-09 14:51:22',
  'expirationDate': '2022-02-18',
  'strike': 75,
  'lastPrice': 100.85,
  'bid': 97.3,
  '

In [5]:
aapl.option.exercise_price

70

In [13]:
OPTIONS["AAPL"]["options_chain"]['2022-02-18']

,Theta,Gamma,Rho,Vega,cName,cDelta,cVolume,cBidP,cTheo,cAskP,Strike,pBidP,pTheo,pAskP,pVolume,pDelta,pName
0,0.0000,0.0000,0,0.0000,AAPL220218C00070000,0.9930,1.0,98.70,NaN,99.05,70.0,0.01,NaN,0.01,17.0,0.0000,AAPL220218P00070000
1,0.0000,0.0000,0,0.0000,AAPL220218C00075000,0.9962,12.0,93.70,NaN,93.95,75.0,0.01,NaN,0.01,722.0,0.0000,AAPL220218P00075000
2,0.0000,0.0000,0,0.0000,AAPL220218C00080000,0.9959,1.0,88.70,NaN,89.20,80.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00080000
3,0.0000,0.0000,0,0.0000,AAPL220218C00085000,0.9955,3.0,83.70,NaN,84.20,85.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00085000
4,0.0000,0.0000,0,0.0000,AAPL220218C00090000,0.9952,1.0,78.70,NaN,79.20,90.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00090000
5,0.0000,0.0000,0,0.0000,AAPL220218C00095000,0.9947,5.0,73.60,NaN,73.95,95.0,0.01,NaN,0.01,45.0,0.0000,AAPL220218P00095000
6,0.0000,0.0000,0,0.0000,AAPL220218C00100000,0.9943,9.0,68.60,NaN,68.95,100.0,0.01,NaN,0.01,100.0,0.0000,AAPL220218P00100000
7,0.0000,0.0000,0,0.0000,AAPL220218C00105000,0.9938,23.0,63.70,NaN,63.95,105.0,0.01,NaN,0.01,1.0,0.0000,AAPL220218P00105000
8,0.0000,0.0000,0,0.0000,AAPL220218C00110000,0.9974,1.0,58.60,NaN,59.20,110.0,0.01,NaN,0.01,12.0,0.0000,AAPL220218P00110000
9,0.0000,0.0000,0,0.0000,AAPL220218C00115000,0.9853,8.0,53.60,NaN,54.05,115.0,0.01,NaN,0.01,2.0,0.0000,AAPL220218P00115000


In [16]:
OPTIONS

{'AAPL': {'code': 'AAPL',
  'exchange': 'US',
  'lastTradeDate': '2022-02-17',
  'lastTradePrice': 168.88,
  'data': [{'expirationDate': '2022-02-18',
    'impliedVolatility': 36.4222,
    'putVolume': 219722,
    'callVolume': 233962,
    'putCallVolumeRatio': 0.9391354151528881,
    'putOpenInterest': 572337,
    'callOpenInterest': 617262,
    'putCallOpenInterestRatio': 0.9272189119045073,
    'optionsCount': 123,
    'options': {'CALL': [{'contractName': 'AAPL220218C00070000',
       'contractSize': 'REGULAR',
       'contractPeriod': 'MONTHLY',
       'currency': 'USD',
       'type': 'CALL',
       'inTheMoney': 'TRUE',
       'lastTradeDateTime': '2022-02-14 15:55:06',
       'expirationDate': '2022-02-18',
       'strike': 70,
       'lastPrice': 98.2,
       'bid': 98.7,
       'ask': 99.05,
       'change': -4.23,
       'changePercent': -0.0413,
       'volume': 1,
       'openInterest': 16,
       'impliedVolatility': 744.2521,
       'delta': 0.993,
       'gamma': 0.0003

In [68]:
# Annualised Log Return (of Underlying)
import math
url = f"https://eodhistoricaldata.com/api/eod/AAPL.US?api_token=OeAFFmMliFG5orCUuwAKQ8l4WWFQ67YX"
response = requests.request("GET", url)
with open('data/AAPL_historical_data.csv', 'w') as f:
    f.write(response.text)
df = pd.read_csv('data/AAPL_historical_data.csv')
dfa = df.dropna().iloc[len(df)-252:len(df)]
sum([math.log(row['Close']/dfa.loc[i-1]['Close']) for i, row, in dfa.iterrows() if i != dfa.iloc[0].name])

0.2940182254412781